In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction
In this notebook, we will show you use the Agent Builder SDK to build a Generative AI Agent + Tools.

## Prerequisites
- Ensure you have a GCP Service Account key with the Dialogflow API Admin privileges assigned to it.

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/dfcx-scrapi/blob/main/examples/bot_building_series/bot_building_101.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/dfcx-scrapi/blob/main/examples/bot_building_series/bot_building_101.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/dfcx-scrapi/blob/main/examples/bot_building_series/bot_building_101.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

In [ ]:
#If you haven't already, make sure you install the `dfcx-scrapi` library
!pip install dfcx-scrapi

import sys

# Colab Auth needed to call Client Endpoints (i.e. vertexai)
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

# Build a New Agent Application
The first thing we will do is create a blank Agent Application.<br>
This is the fundamental building block for chaining together Agents, Tools, and adding few-shot examples.

In [ ]:
PROJECT_ID = "<YOUR_GCP_PROJECT>"

In [ ]:
from dfcx_scrapi.core.agents import Agents

a = Agents()
agent = a.create_agent(
    project_id=PROJECT_ID,
    display_name="SCRAPI AGENTS",
    gcp_region="us-east1",
    playbook_agent=True
)

panel = "(playbooks/00000000-0000-0000-0000-000000000000/basics//right-panel:simulator)"
print(f"AGENT LINK: https://vertexaiconversation.cloud.google.com/{agent.name}/{panel}")

# Test Your Agent
When your Agent application is first created, there is a "Default Generative Agent" created, which has no `goal` or `instructions`.<br>
However, you can still interact with it like a generic LLM model.

One of the great aspects of Vertex Agent Builder is that it automatically handles Session Management for you!<br>
All Agent Applications are ***immediately*** Production scalable and ready, because they are backed by production grade, scalable Google infrastructure.<br>

Want to open you app to 100,000 users immediately?<br>
We've got you covered! And no extra work on your end.

In [ ]:
from dfcx_scrapi.core.sessions import Sessions

s = Sessions()

session_id = s.build_session_id(agent.name)
session_id

In [12]:
res = s.detect_intent(agent.name, session_id, "hey, how are you?")
s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> hey, how are you?

<font color='green'><b>AGENT RESPONSE:</font></b> I'm doing well, thank you! How can I help you today?

In [13]:
res = s.detect_intent(agent.name, session_id, "what kind of models do you have?")
s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> what kind of models do you have?

<font color='green'><b>AGENT RESPONSE:</font></b> We have a variety of models to choose from. What type of vehicle are you interested in?

# Update the Default Playbook
Our Agent isn't very useful at this stage, so let's update the `goal` and `instructions` to do something more worthwhile!<br>

We'll first fetch the handy `playbooks_map` so we can easily refernce IDs which are required by the API endpoints.

In [ ]:
from dfcx_scrapi.core.playbooks import Playbooks

p = Playbooks(agent.name)

playbooks_map = p.get_playbooks_map(agent.name, reverse=True)
playbooks_map

In [16]:
# You see here that there are is no `goal` or `instruction` sets yet.
playbook = p.get_playbook(playbooks_map["Default Generative Agent"])
print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")

GOAL: Default goal
INSTRUCTIONS: 


In [17]:
# We'll provide a simple `goal` and `instruction` set to get started.

playbook = p.update_playbook(
    playbooks_map["Default Generative Agent"],
    goal="You are a friendly Tesla service center agent.\nYour job is to help users book appointments and answer any questions they have.",
    instructions=["Greet the user.", "Answer any questions the have to the best of your ability."]
    )

print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")

GOAL: You are a friendly Tesla service center agent.
Your job is to help users book appointments and answer any questions they have.
INSTRUCTIONS: steps {
  text: "Greet the user."
}
steps {
  text: "Answer any questions the have to the best of your ability."
}



## Test Agent Again
Now that we've updated the `goal` and `instruction` set, let's see how our Agent application reacts.

In [18]:
from dfcx_scrapi.core.sessions import Sessions

s = Sessions()

session_id = s.build_session_id(agent.name)
res = s.detect_intent(agent.name, session_id, "what kind of models do you have?")
s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> what kind of models do you have?

<font color='green'><b>AGENT RESPONSE:</font></b> We have a variety of models to choose from, including the Model S, Model 3, Model X, and Model Y. Which one are you interested in?

Voila! Now we're getting somewhere!<br>
When we update our Agent application's `goal` and `instruction` sets, we are able to steer the Agent towards doing the task we design for it.<br>
Notice the difference in responses when we ask the intentionally ambiguous question about "models".<br>

We can continue to take this further by creating Tools for our Agent to use.

# Create Tools
Tools allow your Agent to interact with the outside world.<br>
There are 3 primary tool types that you can use:
- OpenAPI Spec
- Functions
- Data Stores

In this next section, we'll create an `OpenAPI Spec` tool that can do the following:
- Find the weather using the [National Weather Service Web API](https://www.weather.gov/documentation/services-web-api)

For more information, see [Vertex Agents Tools](https://cloud.google.com/dialogflow/vertex/docs/concept/tools)

## Deploy Cloud Run Endpoints
Using the following code, you can deploy a Cloud Run endpoint to call the National Weather Service API.

In [ ]:
%mkdir get_weather
%cd get_weather

In [19]:
%%writefile requirements.txt
functions-framework==3.*
firebase-admin==6.2.*
firebase-functions
requests==2.*

Writing requirements.txt


In [ ]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/dfcx-scrapi/main/data/get_weather_tool.py
!mv get_weather_tool.py main.py
!gcloud config set project $PROJECT_ID
!gcloud functions deploy get_weather --region "us-central1" --runtime python311 --trigger-http --entry-point main --source main.zip --no-allow-unauthenticated --gen2

## Define OpenAPI Specs
Before proceeding, make sure you replace the URL in the `get_weather_spec` with the CLOUD RUN ENDPOINT that you just deployed!<br>
Be sure to use the CLOUD RUN ENDPOINT, NOT the Cloud Function endpoint!

In [7]:
get_weather_spec = """
openapi: 3.0.0
info:
  title: get_weather
  version: 1.0.0

servers:
  - url: YOUR_ENDPOINT_HERE

paths:
  /get_weather_grid:
    get:
      summary: Returns the current grid information for a city and state
      operationId: get_weather_grid
      parameters:
        - name: latitude
          in: query
          required: true
          schema:
            type: string
        - name: longitude
          in: query
          required: true
          schema:
            type: string
      responses:
        '200':
          description: OK
          content:
            application/json:
              schema:
                type: object
                properties:
                  data:
                    type: string
"""

### Call Create Tool
For creating the `OpenAPI Spec` tool, we'll use the `build_open_api_tool` helper method to create the proper Tool object, then pass it to the `create_tool` method and capture the result.

In [8]:
from dfcx_scrapi.core.tools import Tools

t = Tools()

# Tool 1 - get_weather
weather_tool = t.build_open_api_tool(
    "get_weather",
    spec=get_weather_spec,
    description="Get the current weather for the provided city."
    )
tool1 = t.create_tool(agent.name, weather_tool)

# Create Weather Agent
Now that we have 2 new tools to work with, let's apply them to a new Agent in our application.

In [ ]:
from dfcx_scrapi.core.playbooks import Playbooks

p = Playbooks(agent_id=agent.name)

instructions = [
    "Use the ${TOOL:get_weather} to get the current city/state weather grid information.",
    "- If the user only provides the city you can assume the state unless otherwise specified."]

# New Playbook
weather_agent = p.create_playbook(
    agent.name,
    display_name="Weather Agent",
    referenced_tools=[tool1.name],
    goal="You are a senior weather advisor at a network broadcast station. Your job is to predict the weather!",
    instructions=instructions
)

playbooks_map = p.get_playbooks_map(agent.name, reverse=True)
playbooks_map

## Chain Agents
In order for our Agents to work together properly, we need to chain them together and update the instruction set.

Remember, our current architecture looks like this:
- Default Generative Agent
- Weather Agent

These are mutually exclusive Agents that cannot interact yet.<br>
We want our architecture to look like this instead:
- Default Generative Agent -> Weather Agent

In order to do this, let's update the instruction set on our `Default Generative Agent`.

In [ ]:
p.update_playbook(
    playbooks_map["Default Generative Agent"],
    instructions=["If the user needs help with the weather, call ${AGENT: Weather Agent}"]
)

## Test Agent Again
With our Agents chained together, let's test with a new query about weather in a specific city!

In [11]:
from dfcx_scrapi.core.sessions import Sessions

s = Sessions()

session_id = s.build_session_id(agent.name)
res = s.detect_intent(agent.name, session_id, "what is the weather like in Austin, Texas?")
s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> what is the weather like in Austin, Texas?

<font color='dark red'>TOOL CALL:</font></b> get_weather_grid -> 30.2672

<font color='yellow'>TOOL RESULT:</font></b> {'temperatureUnit': 'F', 'temperature': 77.0}

<font color='green'><b>AGENT RESPONSE:</font></b> The current temperature in Austin, Texas is 77 degrees Fahrenheit.

# Multi Turn Session
Example of a Multi Turn Session with some "cleaner" readability for colab testing.

In [16]:
session_id = s.build_session_id(agent.name)

conversation = [
    "What colors does the model 3 come in?",
    "What's the weather like in Boston right now? I'm thinking about walking to the Tesla dealership."
    ]

i = 1
for utterance in conversation:
  print(f"\n--- TURN {i} --- \n")
  res = s.detect_intent(agent.name, session_id, utterance)
  s.parse_result(res)
  i+=1


--- TURN 1 --- 



<font color='green'><b> USER QUERY:</font></b> What colors does the model 3 come in?

<font color='green'><b>AGENT RESPONSE:</font></b> The Model 3 comes in 5 colors: black, white, blue, red, and gray.


--- TURN 2 --- 



<font color='green'><b> USER QUERY:</font></b> What's the weather like in Boston right now? I'm thinking about walking to the Tesla dealership.

<font color='dark red'>TOOL CALL:</font></b> get_weather_grid -> 42.3601

<font color='yellow'>TOOL RESULT:</font></b> {'temperatureUnit': 'F', 'temperature': 76.0}

<font color='green'><b>AGENT RESPONSE:</font></b> The current temperature in Boston is 76 degrees Fahrenheit.

# GenAI Agents 101 End
In this notebook we've shown how to do the following:
- Create a new Agent Application
- Update the default Agent instructions
- Create a new Tool
- Create a new Agent and apply the new Tool
- Chain together our Agents + Tool to make a simple, cohesive application